# Labsheet 6: Odometry

This worksheet will cover reading the Rotary Encoders on your 3Pi+ robot, and then using this sensor data to maintain an estimate of the robot position on a 2D plane.  

In previous labsheets you will have:
- blah
- blah

In this labsheet we are going to:



## A note on Power  vs. Speed

In the previous labsheet we sent **power** values to the motors, and we observed the **speed**.   Note that, we didn't set or control the speed.  

**It is important to recognise that the Romi robot was not controlling speed**, and it could not know what speed the wheels were rotating at.  Consider, if you applied resistence to the wheel, it slowed down.  To achieve a constant speed, your Romi would need to increase the power to the motor when the wheels are meeting resistance.  This would be the essence of a closed-loop, feedback control system.

In order to begin to autonomously control speed, your Romi must be able to detect the rotation of the wheels with respect to time.  This lab sheet investigates a rotary encoder sensor for detecting rotation (distance).  Labsheet 7 concerns time, and will allow you to calculate and control speed.


<hr><br><br><br><br>

# Overview: Odometry

**`Odometry`** refers to maintaining an estimate of position using sensors.  The 3Pi+ has a rotary encoder per motor, which allows for the detection of rotation.  By using a last-known position and a subsequent measurement of change, it is possible to calculate an estimate of the robot position.  This technique is known as **`dead reckoning`**.

<p align="center">
<img width="75%" src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_InfoSource.png?raw=true">
</p>

In the last Labsheet "Line Following", you will have developed your first `closed-loop` control system.  Hopefully, your robot was able to follow the line sufficiently well.  This would mean your robot appeared to have an intelligent autonomous behaviour.  It is important to recognise that to achieve this, your robot was utilising a `source of information` from the environment - the line.  

In other parts of the line following challenge, this same information is not available.  For instance:
- traversing the gap in the line.
- determining the end of the line.
- returning to home (start position). 

If we can utilise the rotary encoders to estimate the robot position, we can use this information to perform operations such as:
- turn to a specific angle
- move to a specific location
- maintain a straight course of travel
- estimate the distance travelled
- etc.


<hr><br><br><br><br>

# Encoders

The 3Pi+ has a sensor subsystem called a `rotary encoder`, and one of these attached to each motor (two in total).   The rotary encoder is able to indicate when rotation has occured.  Some rotary coders are `absolute`, meaning they can be read to know the exact rotation of the device they are attached to.  Other encoders are `incremental`, meaning that when read they only indicate a change in rotation.  It is also possible to get position encoders which measure linear travel.  

There are two principle ways to read an encoder:
- **`polling`**: if a device is polled, it means the device is read at regular time intervals.  It is important that the read operation happens sufficiently frequently or important information may be missed.  This technique will be demonstrated and discussed as insufficient in one of the unit lectures.
- **`interrupt-driven`**: in interrupt driven subsystems, either the subsystem can issue a flag to indicate new information is ready, or the host computer can set peripheral hardware to monitor for change.  In both cases, an `interrupt` flag (request) is set, and an `interrupt service routine` (ISR) is required to handle the information within an appropriate time.

The 3Pi+ 32u4 microcontroller is able to configure peripheral hardware to monitor for status changes on some of the `gpio` pins. Every time an encoder is moved, it will trigger the `interrupt service routine` (ISR).  The ISR will effectively keep a count of how many times it has been called.  The ISR will also register whether it was a forwards or backwards movement, to increment or decrement the count respectively:

<p align="center">
<br>
<img width="50%" src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_EncoderCount.png?raw=true">
<br>
</p>

## Exercise 1: 3Pi+ Encoders

1. Does the 3Pi+ have `absolute` or `incremental` rotary encoders?  (documentation <a href="https://www.pololu.com/docs/0J83/5.4">1</a>, <a href="https://www.pololu.com/product/3081">2</a>)

2. We could say that the encoders detect the rotation of either the motors (`actuators`), or the wheels (`effectors`).  Which of these (motors/wheels) is the more important description of the nature of the information the 3Pi+ will be able to perceive?

3. The designers of the 3Pi+ attached the encoders to the motors prior to the reduction gearbox.  
  - What is an advantage of this design choice?  
  - What is a disadvantage of this design choice?


<hr><br><br><br><br>

# Quadrature Encoding

For this section we will consider only 1 encoder, which has 2 `hall effect` sensors (`transducers`), and 1 magnet.

<p align="center">
<br>
<img width="75%" src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_EncoderAssembly.png?raw=true">
<br>
</p>

When the wheel is rotated, the magnet is rotated, and so the magnetic field moves over the sensor circuit.  The two `Hall effect` sensors alter their output with the magnetic field, and so two signals are generated.  The state of these two signals can be interpretted as two logic signals.  The combination of two logic signals (either `0` or `1`) provide 4 possible states (base two, two signals: $2^2$), hence the name `quadrature` encoders. 

The two logic signals are typically referred to as **Channel A** and **Channel B**.  Importantly, the **relationship** between these two signals is defined by their physical placement on the sensor circuit.  The relationship between Channel A and B will inform the measurement of rotation.
> "What is meant by relationship?"

Here, the relationship of the two signal means `how they change with respect to one another`.  If they were `independent` variables, it would mean there was no relationship at all, and they could vary independently.  In the case of this rotary encoder, both of the `Hall effect` sensors are exposed to the same varying magnetic field, and both of their positions relative to the field are fixed by the construction of the circuit board.  This means that the Channel A and Channel B signals can be understood to vary relative to one another.  

<p align="center">
<br>
<img width="75%" src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_HallEffect90.png?raw=true">
<br>
</p>

The graphic above is for a <a href="https://www.pololu.com/product/3081">similar sensor circuit</a> as built into the 3Pi+ encoder subsystem.  The graphic has been annotated to indicate where the `rear motor shaft` is located, which is the point of rotation of the magnet.  The two `Hall effect` sensors are annotated, and their position is marked to show that they sit at 90&deg; to one another with respect to the origin of the rotating magnetic field.  This means that their measurement of the magnetic field will **always** be 90&deg; out of phase (out of synchrony, or shifted, by a quarter cycle).

<p align="center">
<br>
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_90Phase.png?raw=true">
<br>
</p>

The above diagram illustates how the magnetic field (north and south poles, red and blue respectively) will move over the two `Hall effect` sensors.  A small red dot has been added to help indicate how the field is rotating across the four states in time, read left to right respectively.  In reality, the magnet will move continuously, but these are the essential 4 states registered by the encoder.

The logic signals for Channel A and B are visible above as  the `square waveforms`, and coloured to match the magnetic field polarity detected in each state.  These waveforms will endlessly repeat their high-low-high transitions as the magnet is rotated above them.  Notice that, if we mark in time where each Channel transitions to logic `1` (or high), we can see they are 90&deg; out of phase with respect to each other.

|Channel B   | Channel A   |
|:---:|:---:|
| 0  | 0  |
| 0  | 1  |
| 1  | 1  |
| 1  | 0  |

We can examine the above diagram and produce the above logic table, where south-polarity (blue) is logic 0, and north-polarity (red) is logic 1.  These are the 4 possible states of the `hall effect` sensors combined.  If the magnet continues rotating in the same direction as the illustration, this logic table would repeat itself.  At this point, we could be satisfied by registering a logic change, to keep a "count" of how many times the encoder has detected a rotation.  However, we know that the wheels of the 3Pi+ can be rotated in either direction.

<p align="center">
<br>
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_EncoderXORSignals.png?raw=true">
<br>
</p>

It is therefore important to register the `logic transitions` in the signals.  In the above illustration, the transitions of the waveforms have been annotated.  Note that, the transitioning logic has an arrow next to it - this is because the progression of these waveforms could be read left-to-right, or right-to-left, depending on which way the wheel is rotating.  We will use the logic of this direction information later in "Example Encoder Code". 

In order to capture a transition from low-to-high, or high-to-low, it is necessary to have `prior` information (what was the state of the channel before?).  

<p align="center">
<br>
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_EncoderDirection.png?raw=true">
<br>
</p>

In the above illustration, the `prior` state of the encoder is marked in red, as `A = 1, B = 1`.  Our code would then need to work out if it is either **channel A** which changes, or **channel B** which changes, away from the prior state.  This "direction" of this transition away from the prior state will allow our robot to count up or down with respect to the direction of travel:

<p align="center">
<br>
<img width="50%" src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_EncoderCount.png?raw=true">
<br>
</p>





<hr><br><br><br><br>

# Pin Change Interrupts

In the previous section, we have discussed how the 3Pi+ can determine direction of rotation by registering the transition of the A and B channels of the encoder.  We have used the assumption that an encoder `interrupt service routine` (ISR) will activate when the wheel is moved.  This is achieved on the 3Pi+ by using a **`Pin Change Interrupt`**.  

**`Pin Change Interrupts`** are a special piece of hardware.  Microcontrollers have many useful built-in peripherals.  A pin change interrupt watches a specific pin on the chip for a change of state (low to high, or high to low).  When the microcontroller detects a change, it can be set to automatically run a piece of code.  The piece of code which runs automatically is called an **`Interrupt Service Routine`** (ISR).  An ISR is not exclusively for pin changes - they can be triggered to run by many peripherals, taking input internal or external to the microcontroller.  

An `interrupt` is so called because it will stop (pause) sequence of your main code.  These are especially useful to catch external events, rather than `polling` to detect them.  However, most microcontrollers have only one central processing unit (CPU, often called a 'core').  Therefore, because your code within `loop()` runs on the only CPU inside the 32u4, it will be paused to run an ISR instead.  

You can think of an ISR as being a background task which runs "only when it needs to".  However, you are responsible for determining the cause of the interrupt (what triggers it), and the efficiency of the ISR (how much CPU time it will use).  

> As a general rule, code within ISR's should be as short as possible in order to avoid taking up too much CPU time.

If you are experimenting with an ISR and your robot starts to behave in a strange way, you may have made your ISR too big (or more seriously, have a bug within the ISR code).  Note that, using ISR's means the execution of your code is no longer `deterministic` - the compiler cannot predict when an ISR will activate, and neither can you.  For example, we cannot know when a wheel and motor will rotate.  

Obviously, a Pin Change Interrupt is very convenient for watching the state of the encoders.  We can consider catching encoder changes to be a high priority, perhaps higher than our main program.  For example, if we miss encoder changes, we will lose track of our robot position.  

## A Slight Complication

A limitation is that only specific pins can be monitored by a Pin Change Interrupt, and there are a finite number of pins to use.  The 3Pi+ has two encoders, each with two pins.  This means that in an ideal world we might like to watch 4 independent pin changes in total.  

The designers of 3Pi+ decided to use some electronics to use just 1 pin per encoder to trigger a pin change interrupt - using just one pin to register a change for either pins to an encoder.  They have achieved this by using an electronic XOR (eXclusive OR) logic gate across the A and B channels, one XOR for each encoder.  

Recall that the signals for A and B are always out of phase by 90&deg;.  Therefore, by XOR'ing A with B, we can register when a change occurs to either A or B through just one pin (it will always be exclusively one or the other that changed).  

This electronic XOR result is sent through a single Pin Change Interupt pin, whilst the normal result for the channel B encoder pin can be read with a normal digitalRead().  By using a software-XOR operation to compare the pin change interrupt source and Channel B, the state of Channel A can be identified.  You can find more information in the <a href="https://www.pololu.com/docs/0J83/5.4">documentation</a>

**For now, you don't need to fully understand this operation - it is a matter of interest**.  If you want to fully understand it, I suggest you draw out a logic truth table for XOR, then some pin transitions, and apply XOR logic.  In any case, you'll notice the following section of code within our ISRs to perform this XOR and restore the A and B pin states.  It looks a little strange - **but don't worry, it is complete and you do not need to modify it:**

```c
  // We know that the ISR is only called when a pin changes.
  // We also know only 1 pin can change at a time.
  // The XOR(AB) signal change from "Channel A" triggers ISR.
  
  // First, Read in the new state of the encoder pins.
  // Standard pins, so standard read functions.
  boolean e0_B = digitalRead( E0_B_PIN ); // normal B state
  boolean e0_A = digitalRead( E0_A_PIN ); // XOR(AB)

  // Software XOR (^) logically infers
  // the true value of A given the state of B
  e0_A = e0_A ^ e0_B;
```
**You can just assume the above XOR operation works, and simply make sure you do not delete it from the template code provided.**





<hr><br><br><br><br>

# Setting Up Pin Change Interrupts

Configuring the microcontroller hardware to use the Pin Change Interrupts is a little bit complicated, so we have done this for you.  You can find this code within `encoders.h` of the Arduino code template on Github (<a href="https://github.com/paulodowd/EMATM0053_21_22/tree/main/3PI_CodeStub">page</a>, <a href="https://github.com/paulodowd/EMATM0053_21_22/blob/main/3PI_CodeStub/Labsheet_X.zip">Zip file</a>).  For further information you can look at the two functions:
- **setupEncoder0()**
- **setupEncoder1()** 

These set up Pin Change Interrupts for the two encoders on the  3Pi+.  Both functions are heavily commented.  Both involve bitwise operations to registers within the microcontroller.  Setting bits in registers activates peripherals or configures the microcontroller - this should have been covered in a lecture you have received.  If you'd like to experiment with similar operations, it is recommended you look at the labsheet concerning the Timer peripheral.  


<hr><br><br><br><br>

# The Two Interrupt Service Routines (ISRs)

An ISR is written to service an event from a particular peripheral.  This is achieved by using a special function declaration, which attaches to the source of the interrupt (a good start for further reading is available <a href="https://en.wikipedia.org/wiki/Interrupt_vector_table">here</a>).  In our case, we are using pins which carry the definition `INT6_vect` and `PCINT0_vect`, there are where the encoders have been wired too on the 3Pi+ (<a href="https://www.pololu.com/docs/0J83/5.9">documentation</a>) - and which we can find are Pin Change enabled in the 32u4 microcontroller manual (<a href="https://ww1.microchip.com/downloads/en/DeviceDoc/Atmel-7766-8-bit-AVR-ATmega16U4-32U4_Datasheet.pdf">documentation</a>).  

You'll find these details referenced in the setup routines setupEncoder0() and setupEncoder1(). You can find this code within `encoders.h` of the Arduino code template on Github (<a href="https://github.com/paulodowd/EMATM0053_21_22/tree/main/3PI_CodeStub">page</a>, <a href="https://github.com/paulodowd/EMATM0053_21_22/blob/main/3PI_CodeStub/Labsheet_X.zip">Zip file</a>).  Our ISR function declarations look like:

```c
ISR( INT6_vect ) {
 // ....
 // ...ISR code - keep it short!...
 // ...A basic template is provided for you.
 // ....
}
ISR( PCINT0_vect ) {
 // ....
 // ...ISR code - keep it short!...
 // ...A basic template is provided for you.
 // ....
}
```

After calling the `setupEncoder0()` and `setupEncoder1()` inside `setup()`, these two ISR functions will execute whenever the encoder changes state.  
> Therefore, you never need to call `ISR( ... )` explicitly.

Note that the functions cannot take any arguments or provide any return type.  

To have any persistent data, these ISR's will have to operate on `global` variables.  Because we don't know when the ISR's will execute (e.g., we cannot predict when the wheel will rotate), we also need to declare the associated global variables as `volatile`.   Volatile is a keyword to the compiler, informing the compiler not to use any cache based optimatisations.  Therefore, in global scope, we declare the following global variables to be used by our ISR functions:

```c
// Volatile Global variables used by Encoder ISR.
volatile long count_e1; // used by encoder to count the rotation
volatile byte state_e1; // used to store the prior and current state.

volatile long count_e0; // used by encoder to count the rotation
volatile byte state_e0; // used to store the prior and current state
```

For the remaining implementation details for the Pin Change Interrupt ISR's, you can read the comments provided in the example code.



<hr><br><br><br><br>

# Example Encoder Code

Now that we understand the logical operation of the encoders, and we understand that they will be read by interrupt service routines, it is time to take a closer look at code.  The Arduino code template on Github (<a href="https://github.com/paulodowd/EMATM0053_21_22/tree/main/3PI_CodeStub">page</a>, <a href="https://github.com/paulodowd/EMATM0053_21_22/blob/main/3PI_CodeStub/Labsheet_X.zip">Zip file</a>) provides template functions within  `encoders.h` which you will need to complete.  This section discusses how the code within `encoders.h` will operate.

## Decoding Encoders

From previous sections we know that:
- the ISR will activate when either Channel A or Channel B changes state.
- we can read and decode the correct current state of A and B when the ISR is activated.
- we need to register which channel has changed to infer the `direction` of change.
- therefore, we need the prior state and the current state of the encoder.

We can consider that the system needs to maintain 4-bits of information:
- The `prior` states $(t-1)$ of Channel A (0/1) and B (0/1)
- The `current` states $(t)$ of Channel A (0/1) and B (0/1)

Because the states of A and B are boolean logic, the theoretical number of possible combinations is $2^4$, or 16 possible states.

|Index| Channel B (t)  | Channel A (t)  |-| Channel B (t-1)  | Channel A (t-1)  | Notes  |
|:---:|:---:|:---:|-|:---:|:---:|:---|
| 0 | 0  | 0  |-|  0 |  0 |  (no change, invalid) | 
| 1 | 0  | 0  |-|  0 | 1  |   |
| 2 | 0  | 0  |-|  1 |  0 |  |
| 3 | 0  | 0  |-|  1 |  1 | (change x2, invalid) |
| 4 | 0  | 1  |-|  0 |  0 |  |
| 5 | 0  | 1  |-|  0 |  1 | (no change, invalid)|
| 6 | 0  | 1  |-|  1 |  0 | (change x2, invalid) |
| 7 | 0  | 1  |-|  1 |  1 |  |
| 8 | 1  | 0  |-|  0 |  0 |  |
| 9 | 1  | 0  |-|  0 |  1 |  (change x2, invalid)|
| 10 | 1  | 0  |-|  1 |  0 |  (no change, invalid)|
| 11 | 1  | 0  |-|  1 |  1 |  |
| 12 | 1  | 1  |-|  0 |  0 |  (change x2, invalid)|
| 13 | 1  | 1  |-|  0 |  1 |  |
| 14 | 1  | 1  |-|  1 |  0 |  |
| 15 | 1  | 1  |-|  1 |  1 | (no change, invalid) |

The above logic table illustrates all the theoretical states of the encoders.  From our prior undestanding of the encoder operation, we can invalidate several possible states immediately, as annotated above.  This leaves 8 entries in the table as valid.

|Index| Channel B (t)  | Channel A (t)  |-| Channel B (t-1)  | Channel A (t-1)  | Notes  |
|:---:|:---:|:---:|-|:---:|:---:|:---|
| 0 | 0  | 0  |-|  0 |  0 |  (no change, invalid) | 
| 1 | 0  | 0  |-|  0 | 1  |  +1 |
| 2 | 0  | 0  |-|  1 |  0 |  -1 |
| 3 | 0  | 0  |-|  1 |  1 | (change x2, invalid) |
| 4 | 0  | 1  |-|  0 |  0 | -1 |
| 5 | 0  | 1  |-|  0 |  1 | (no change, invalid)|
| 6 | 0  | 1  |-|  1 |  0 | (change x2, invalid) |
| 7 | 0  | 1  |-|  1 |  1 | +1 |
| 8 | 1  | 0  |-|  0 |  0 | +1 |
| 9 | 1  | 0  |-|  0 |  1 |  (change x2, invalid)|
| 10 | 1  | 0  |-|  1 |  0 |  (no change, invalid)|
| 11 | 1  | 0  |-|  1 |  1 | -1 |
| 12 | 1  | 1  |-|  0 |  0 |  (change x2, invalid)|
| 13 | 1  | 1  |-|  0 |  1 | -1 |
| 14 | 1  | 1  |-|  1 |  0 | +1 |
| 15 | 1  | 1  |-|  1 |  1 | (no change, invalid) |

We can use the logic as presented in <a href="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_EncoderDirection.png?raw=true">this illustration</a> to complete the table above with a direction of rotation.  In the table above, `+` or `-` are used to indicate direction.  Whether this corresponds to forwards or backwards motion of your 3Pi+ will need to be `empirically` validated.

If we look at the table above, we can imagine writing an if(){} statement to select the appropriate table row given our encoder states, something like:

```c
    if( (old_b == 1) && (old_a == 0) && (new_b == 0) && (new_a == 0) ) {        // Row 1
        
    } else if( (old_b == 0) && (old_a == 1) && (new_b == 0) && (new_a == 0) ) { // Row 2
        
    } else if( ... ) { // ... etc

```
However, this is very long winded, and we will need to maintain lots of different variables.


If we consider the table above, we can see that the valid entries have the index values: `1`, `2`, `4`, `7`, `8`, `11`, `13` and `14`.  These values are not assigned randomly or by chance.  In fact, if we relabelled the columns of the table with bitwise magnitude values, it becomes apparent the `index` is the decimal value of the columns as binary representation:

|Decimal| 8:  Channel B (t)  | 4: Channel A (t)  |-| 2: Channel B (t-1)  | 1: Channel A (t-1)  | Notes  |
|:---:|:---:|:---:|-|:---:|:---:|:---|
| 0 | 0  | 0  |-|  0 |  0 |  (no change, invalid) | 
| 1 | 0  | 0  |-|  0 | 1  |  +1 |
| 2 | 0  | 0  |-|  1 |  0 |  -1 |
| 3 | 0  | 0  |-|  1 |  1 | (change x2, invalid) |
| 4 | 0  | 1  |-|  0 |  0 | -1 |
| 5 | 0  | 1  |-|  0 |  1 | (no change, invalid)|
| 6 | 0  | 1  |-|  1 |  0 | (change x2, invalid) |
| 7 | 0  | 1  |-|  1 |  1 | +1 |
| 8 | 1  | 0  |-|  0 |  0 | +1 |
| 9 | 1  | 0  |-|  0 |  1 |  (change x2, invalid)|
| 10 | 1  | 0  |-|  1 |  0 |  (no change, invalid)|
| 11 | 1  | 0  |-|  1 |  1 | -1 |
| 12 | 1  | 1  |-|  0 |  0 |  (change x2, invalid)|
| 13 | 1  | 1  |-|  0 |  1 | -1 |
| 14 | 1  | 1  |-|  1 |  0 | +1 |
| 15 | 1  | 1  |-|  1 |  1 | (no change, invalid) |

We can then write an `if()` statement to check the decimal value, knowing it corresponds to a pair of `prior state` and `current state` of the encoder.  

```c
if( state_e0 == 1 ) {
  count_e0 = count_e0 + 1;  // forwards

} else if( state_e0 == 2 ) {
  count_e0 = count_e0 -1;   // backwards

} else if () { //...etc
```

The task at hand therefore becomes storing the `prior state` and `current state` information in a binary format to read as a decimal value from a single variable.  You may have noticed earlier we declared a variable of datatype `volatile byte` called `state_e0`:

```c

volatile long count_e0; // used by encoder to count the rotation
volatile byte state_e0; // used to store the prior and current state

//...

void setupEncoder0() {

    // Setup pins for encoder 0
    pinMode( ENCODER_0_A_PIN, INPUT );
    pinMode( ENCODER_0_B_PIN, INPUT );

    // initialise the recorded state of e0 encoder.
    state_e0 = 0;

    // Get initial state of encoder pins A + B
    boolean e0_A = digitalRead( ENCODER_0_A_PIN );
    boolean e0_B = digitalRead( ENCODER_0_B_PIN );
    e0_A = e0_A ^ e0_B;

    // Shift values into correct place in state.
    // Bits 1 and 0  are prior states.
    state_e0 = state_e0 | ( e0_B << 1 );
    state_e0 = state_e0 | ( e0_B << 0 );

    // ...
}

```

In the above `setupEncoder0()` routine we can observe how the initial states of the A and B channels are read and stored into bits 0 and 1 of the variable `state_e0`.  The `setupEncoder0()` and `setupEncoder1()` functions should be run when your robot first activates, before any motion.   In this line:

```c
state_e0 = state_e0 | ( e0_B << 1 );
```

The value `e0_B` is being shifted once to the left (`<< 1`), and then logically OR'd (`|`) into the existing value of state_e0.  We can think of this as moving the boolean value of `e0_B` into column 1 of the table.  For more information on how this is working, it is recommended to review bitwise operators (<a href="https://www.arduino.cc/reference/tr/language/structure/bitwise-operators/bitwiseand/">Arduino Reference</a>).  

 

We can then review the ISR function for encoder 0:

```c

// This ISR handles just Encoder 0
// ISR to read the Encoder0 Channel A and B pins
// and then look up based on  transition what kind of
// rotation must have occured.
ISR( INT6_vect ) 
  // We know that the ISR is only called when a pin changes.
  // We also know only 1 pin can change at a time.
  // The XOR(AB) signal change from "Channel A" triggers ISR.
 
  // First, Read in the new state of the encoder pins.
  // Standard pins, so standard read functions.
  boolean e0_B = digitalRead( E0_B_PIN ); // normal B state
  boolean e0_A = digitalRead( E0_A_PIN ); // XOR(AB)
 
  // Software XOR (^) logically infers
  // the true value of A given the state of B
  e0_A = e0_A ^ e0_B;

  // Shift our (new) current readings into bit positions
  // 2 and 3 in the state variable (current state)
  // State: (bit3)  (bit2)  (bit1)   (bit0)
  // State:  new B   new A   old B   old A
  state_e0 = state_e0 | ( e0_B  << 3 );
  state_e0 = state_e0 | ( e0_A  << 2 );

  // Handle which transition is registered:
  if( state_e0 == 2 ) {
  // ...
  // (large segment of code cut)
  // ...

  // Shift the current readings (bits 3 and 2) down
  // into position 1 and 0 (to become prior readings)
  // This bumps bits 1 and 0 off to the right, "deleting"
  // them for the next ISR call.  
  state_e0 = state_e0 >> 2;
}

```




Everytime the motor physically rotates, it will generate a new transition on the encoder.  Typically, encoders refer to this property as **Count Per Revolution (CPR)**, **Pulses Per Revolution (PPR)** or **Lines Per Revolution (LPR)**.  The higher the CPR, the more precisely  rotation can be determined. This is because we are essentially dividing a full rotation (360 degrees, or 2\*PI radians) by the CPR.  One thing that will effect this, is whether the encoder is placed before or after any gearing on the motor.  It may be that a full rotation of the encoder is only a fraction of a full rotation of the wheel.  This is the case for your Romi.  Your Romi has a 120:1 gearbox.  


## Exercise 2: Complete the Encoder ISRs

1. Given we are using the datatype `long` to store the encoder counts, what is the maximum and minimum values that can be stored? 

2. When we update the byte variables `state_e1` and `state_e0` to move `current state` into `prior state` at the end of the ISR, we shift to the right `>>`.  We could imagine writing the whole ISR operation to shift `current state` into `prior state` to the left (`<<`) instead.  What would be the problem with this?  

3. Review the template provided for you in `encoders.h`:
  - You do not need to change anything in `setupEncoder0()` or `setupEncoder1()`.
  - **Decompose the problem:** Work on one encoder ISR to start with.  Once you have one encoder ISR working, work to complete the second encoder.  
  - Complete the ISR `if()` statement to properly increment and decrement the count value.  
    - Many required conditional statements are missing.
    - You can refer to the tables presented above for assistance.
  - **Validate:** Ensure that you increment and decrement `count_e0` and `count_e1` within the correct ISR routines.
    - A common error is that one ISR is updating the other's count.

  - **Validate:** Use `Serial.print` within `loop()` to check that your encoder values can consistently go up and down, with forward and backward motion.
    - Ensure you add the function calls `setupEncoder0()` and `setupEncoder1()` to your `setup()` routine.
    - Should forward motion increment values, or decrement values?  There is no correct answer to this question.  Please decide for your implementation.
  - **Refactor:** The functions and variables have been consistently named `e0` and `e1`:
    - determine which is left and right.
    - consider updating the code to rename e0 and e1 to left/right, or l/r, as appropriate.
    - why might this be useful to do?

<hr><br><br><br><br>

# Exercise 2: Validating it works properly


- Download the code and use the Plotter to check if the values of count_e1 and count_e0 go up and down, depending on which way you rotate the wheel.  


- Rename the variables in the example code so that they mean more to you.  **This will help you in future lab sheets.** You may wish to consider:
    - Is encoder E1 the left or right motor? 
    - Is encoder E0 the left of right motor?
    - Which way would you like your robot to drive forwards? 
    - When the value of count_e1 or count_e0 goes up, does this represent foward or clockwise motion?

# Exercise 3: Exploring Utility (optional)

- Add your own code so that your robot will drive forward for +2000 encoder counts on both wheels and then stop.
    - Improve your code to get your robot to stop as near as possible on a count of +2000 for both wheels.  What are the main factors which make this difficult?
    - Improve your code further to get your robot to draw back to a count of 0 on both left and right wheels.  What are the main factors which make this difficult?
    
- Add your own code so that your robot can turn 90 degrees. 


  

# Exercise 4: Understanding Operation (optional)

- Determine how many encoder counts are required on both wheels to drive forward 10cm.
    - Complete this exercise experimentally with your robot (try and find it using a systematic method - e.g. calibrate!).  
    - Calculate the number of encoder counts that this should be, based on the gear box, the encoder counts per revolution, and the wheel radius.  The following have the necessary information you will need:
        - https://www.pololu.com/product/3542
        - https://www.pololu.com/product/3675/specs
    - Does your calculated encoder counts per wheel revolution match your calibrated value?
        
    
    
    

# Exercise 5: Basic Speed Control (optional):

- Create variables to store the **change of encoder counts** over time (or iteration) for the left and right motor respectively.
    - Have your code aim to maintain a fixed change of encoder counts by varying the power sent to the motor.
    - Test the performance your code by placing your robot on to different driving surfaces (e.g., not on a surface, carpet anda  table top).
  
  
- Evaluate this speed control to allow your Romi to drive in a straight line.  
    - Do you get better "straight line" performance than open loop control?
    - Do you get better "straight line" performance than controlling by strict encoder counts for left and right motors?
    